In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_file = '../body_vectors_drags.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_56,dim_57,dim_58,dim_59,dim_60,dim_61,dim_62,dim_63,dim_64,drag
61,-6.639910,-2.575970,-4.333867,0.348634,-6.591602,4.137893,3.448946,9.801783,-4.944950,-1.340411,...,4.211274,0.876610,2.535916,4.460216,7.625662,5.399125,2.165930,1.335564,-6.787674,0.375
354,5.044733,2.543681,-3.247805,-5.378260,0.549456,-1.075264,6.259791,5.443143,1.828774,5.774058,...,5.602760,-0.438914,-1.589318,4.962416,2.475589,-6.530942,-0.353212,1.797986,-2.464886,0.374
358,-0.939149,0.704528,0.749587,-1.643254,-3.556550,0.293420,3.039790,-3.084175,-3.810488,3.759489,...,-4.860520,-0.330673,2.862438,-0.245570,2.726983,-3.573323,-3.095024,0.883057,-0.973858,0.435
275,-1.177986,1.175732,-1.834498,1.306601,-0.493694,1.298445,2.197183,4.541283,1.859666,3.284920,...,-2.512394,-5.162504,0.902922,7.131485,-3.511894,4.393147,-5.885215,0.389388,5.135883,0.437
18,0.390329,-1.530953,-2.521950,-4.864147,-0.742480,0.146031,-0.156536,1.740131,-2.943921,5.280754,...,1.704322,5.339616,3.019771,6.336881,1.606373,-2.136702,-7.156015,3.795955,1.753440,0.367


In [2]:
train_drags = train_df["drag"]
train_index = train_df["i"]
for index, drag in zip(train_index, train_drags):
    print(index, "," ,drag)

61 , 0.375
354 , 0.374
358 , 0.435
275 , 0.437
18 , 0.367
107 , 0.36
57 , 0.456
430 , 0.386
374 , 0.345
179 , 0.345
287 , 0.341
51 , 0.55
395 , 0.48
133 , 0.371
233 , 0.427
210 , 0.478
151 , 0.294
375 , 0.411
24 , 0.538
311 , 0.356
152 , 0.429
369 , 0.478
8 , 0.317
90 , 0.439
47 , 0.418
305 , 0.423
421 , 0.377
253 , 0.418
167 , 0.322
315 , 0.355
345 , 0.489
366 , 0.42
41 , 0.51
359 , 0.453
269 , 0.362
56 , 0.375
193 , 0.408
283 , 0.513
204 , 0.315
224 , 0.333
316 , 0.571
19 , 0.375
150 , 0.33
118 , 0.298
58 , 0.341
223 , 0.359
1 , 0.393
226 , 0.403
124 , 0.339
232 , 0.404
278 , 0.404
342 , 0.344
26 , 0.4
390 , 0.326
379 , 0.417
81 , 0.384
236 , 0.432
100 , 0.374
383 , 0.315
148 , 0.334
3 , 0.337
371 , 0.329
123 , 0.446
231 , 0.348
248 , 0.37
39 , 0.379
120 , 0.439
273 , 0.317
29 , 0.436
378 , 0.371
352 , 0.429
154 , 0.362
407 , 0.347
131 , 0.357
261 , 0.387
262 , 0.402
382 , 0.468
10 , 0.425
88 , 0.402
372 , 0.445
73 , 0.417
234 , 0.516
106 , 0.307
213 , 0.334
364 , 0.356
206 , 0.434
3

In [3]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    351.000000
mean       0.398513
std        0.060013
min        0.278000
25%        0.353000
50%        0.394000
75%        0.435000
max        0.598000
Name: drag, dtype: float64


In [4]:
%time
import os
import time
start_time = time.time()

save_path = './agModels-body2'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10], 5
bag_sets = 3 #suggestion range [1, 20], 3
stack_levels = 3 #suggestion range [0, 3], 3
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)
end_time = time.time()


Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
Saving ./agModels-body2/learner.pkl
Saving ./agModels-body2/predictor.pkl
Beginning

CPU times: user 5 µs, sys: 3 µs, total: 8 µs
Wall time: 16.5 µs


		Skipping TextNgramFeatureGenerator: No input feature with required dtypes.
		Skipping IdentityFeatureGenerator: No input feature with required dtypes.
		Skipping IsNanFeatureGenerator: No input feature with required dtypes.
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
			Types of features in original data (raw dtype, special dtypes):
				('float', []) : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]
			Types of features in processed data (exact raw dtype, raw dtype):
				('float64', 'float') : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]
			Types of features in processed data (raw dtype, special dtypes):
				('float', []) : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]
			0.0s = Fit runtime
			64 features in original data used to generate 64 features in processed data.
	Types of features in original data (exact raw dtype, raw dtype):
		('float64', 'float') : 64 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]
	Types of features in ori

In [5]:
print(f"Execution time: {end_time-start_time}")

Execution time: 567.7022740840912


In [6]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_55,dim_56,dim_57,dim_58,dim_59,dim_60,dim_61,dim_62,dim_63,dim_64
46,3.262418,4.623484,5.207058,1.474604,5.149589,1.366774,7.350364,4.988407,-4.451728,2.715955,...,3.239450,0.271440,-1.450751,6.201051,2.322523,-0.651795,-2.503192,-1.329733,5.614099,4.784914
101,-1.815653,0.467063,-0.713573,-2.519432,-6.833046,3.925573,3.131740,2.785505,-0.169789,3.797550,...,8.718956,-4.833921,-1.387358,1.010315,0.467406,-7.262593,0.406025,-4.430325,-4.493312,3.210908
175,-0.128397,-0.347771,-4.491439,-0.143762,-1.139696,1.523660,4.283424,1.275272,-3.277825,-0.944104,...,2.371663,-0.860736,-2.739771,4.489244,4.020984,1.973119,3.365454,-2.144289,1.310637,-1.411514
9,-0.334880,-0.358754,-5.121314,-1.869275,0.053525,2.998930,-0.482418,1.283854,-1.355309,-0.988547,...,4.671710,-1.422821,0.833765,5.060172,3.785773,1.105779,2.158277,-2.020028,-0.746978,1.548296
136,0.339348,2.649365,-1.346519,0.872298,-2.672357,1.983254,2.075392,6.573480,-0.809723,-2.231186,...,0.290699,-0.446408,-5.342670,5.205126,5.651273,-2.507123,1.437673,-3.640510,-1.535356,4.507751


In [7]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_5040.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-body2/predictor.pkl
Loading: ./agModels-body2/learner.pkl
Loading: ./agModels-body2/models/trainer.pkl
Loading: ./agModels-body2/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-body2/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-body2/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-body2/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-body2/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-body2/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-body2/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-body2/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-body2/models/CatBoost_BAG_L2/model.pkl
Loading: ./agModels-body2/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-body2/models/LightGBMLarge_BAG_L2/model.pk

0.38182002305984497
0.37923797965049744
0.4182226359844208
0.37530264258384705
0.389003723859787
0.4066256582736969
0.4165061116218567
0.4538293182849884
0.4196510910987854
0.362356573343277
0.3968256711959839
0.41896840929985046
0.3744400143623352
0.4289165735244751
0.4121803045272827
0.40932202339172363
0.37454378604888916
0.3750230669975281
0.3639153838157654
0.35790133476257324
0.4190674424171448
0.41342127323150635
0.37711167335510254
0.39808857440948486
0.44223451614379883
0.39511027932167053
0.4299553632736206
0.38873857259750366
0.4420740604400635
0.39193785190582275
0.41486668586730957
0.36348527669906616
0.3784347176551819
0.4235031306743622
0.39455199241638184
0.38091856241226196
0.37766367197036743
0.37039682269096375
0.4021477997303009
0.3952903151512146
0.42124688625335693
0.35644111037254333
0.4479696452617645
0.39453744888305664
0.4159390926361084
0.40124669671058655
0.39609000086784363
0.3899824023246765
0.35720157623291016
0.35861170291900635
0.3929234743118286
0.3617

Loading: ./agModels-body2/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-body2/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-body2/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-body2/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-body2/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-body2/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-body2/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-body2/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-body2/models/WeightedEnsemble_L2/model.pkl
Loading: ./agModels-body2/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-body2/models/LightGBM_BAG_L2/model.pkl
Loading: ./agModels-body2/models/RandomForestMSE_BAG_L2/model.pkl
Loading: ./agModels-body2/models/CatBoost_BAG_L2/model.pkl


                     model  score_test  score_val  pred_time_test  \
0   NeuralNetFastAI_BAG_L2   -0.048294  -0.049635        8.151326   
1   NeuralNetFastAI_BAG_L3   -0.048798  -0.048729       15.399192   
2           XGBoost_BAG_L4   -0.048865  -0.051141       22.336500   
3      WeightedEnsemble_L3   -0.048937  -0.048836        8.321857   
4   NeuralNetFastAI_BAG_L4   -0.048969  -0.050780       22.420778   
5          LightGBM_BAG_L4   -0.049015  -0.050518       24.038302   
6      WeightedEnsemble_L4   -0.049163  -0.048415       15.758999   
7        LightGBMXT_BAG_L4   -0.049374  -0.050233       24.090878   
8      WeightedEnsemble_L5   -0.049376  -0.049785       28.562546   
9           XGBoost_BAG_L2   -0.049390  -0.053570        8.070613   
10    LightGBMLarge_BAG_L4   -0.049455  -0.050714       25.274370   
11  RandomForestMSE_BAG_L4   -0.049468  -0.050841       22.284206   
12    ExtraTreesMSE_BAG_L3   -0.049661  -0.050695       15.290136   
13         CatBoost_BAG_L4   -0.04

In [8]:
best_model = predictor.get_model_best()
print(best_model)

WeightedEnsemble_L4


In [9]:
train_data_pred = predictor.predict(train_data, model=best_model)
test_data_pred = predictor.predict(test_data, model=best_model)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt('./body_vectors_y_test_hat_sdf.csv', test_data_pred, delimiter=',')
np.savetxt('./body_vectors_y_train_hat_sdf.csv', train_data_pred, delimiter=',')

Loading: ./agModels-body2/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-body2/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-body2/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-body2/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-body2/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-body2/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-body2/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-body2/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-body2/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-body2/models/CatBoost_BAG_L2/model.pkl
Loading: ./agModels-body2/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-body2/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-body2/models/LightGBMXT_BAG_L2/model.pkl
Loading: ./agModels-body2/models/LightGBM_BAG_L2/model.pkl
L